In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm
from robust_optimization import RobustOptimizer
import copy
import timm
import torch.utils.data as data

print(f"Torch: {torch.__version__}")

env: CUDA_VISIBLE_DEVICES=0


C:\Users\Darkl\anaconda3\envs\mtcnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Darkl\anaconda3\envs\mtcnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


Torch: 1.10.2


In [2]:
# Training settings
batch_size = 32
epochs = 40
lr = 3e-5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)
CUDA_LAUNCH_BLOCKING=1
print(torch.cuda.is_available())

training_size = 28317 + 3541
IMG_SIZE=260
image_path = '../dataset/images.npy'
training_emotion_path = "../dataset/emotions_multi.npy"
test_emotion_path = '../dataset/emotions_multi.npy'

True


In [3]:
def load_data(image_path, training_emotion_path, subset):
    images = np.load(image_path)        # shape = (35393, 48, 48, 1)
    # images = images/255.0
    images = np.float32(images)
    
    if subset == 'train':
        training_emotions = np.load(training_emotion_path)
        training_emotions = np.float32(training_emotions)
        random_training_emotions = []
        for i in range(training_size):
            random_emotion_index = np.random.choice(8, p=training_emotions[i]/np.sum(training_emotions[i]))
            emo_lst = [0, 0, 0, 0, 0, 0, 0, 0]
            emo_lst[random_emotion_index] = 1
            emo_lst = np.float32(emo_lst)
            random_training_emotions.append(emo_lst)
        return images[:training_size], np.array(random_training_emotions)
    
    if subset == 'test':
        test_emotions = np.load(test_emotion_path)
        test_emotions = np.float32(test_emotions)
        return images[training_size:], test_emotions[training_size:]

def img_proc(img):
    img = torch.tensor(img)                             # (48, 48, 1)
    img = torch.reshape(img, (1, 48, 48))               # (1, 48, 48)
    img = transforms.Resize([IMG_SIZE, IMG_SIZE])(img)  # (1, 260, 260)
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = img.repeat(3, 1, 1)                           # (3, 260, 260)
    return img

class FERPlusDataset(data.Dataset):
    def __init__(self, image_path, training_emotion_path, subset):
        assert(subset=='train' or subset=='test')
        self.images, self.emotions = load_data(image_path, training_emotion_path, subset)

    def __getitem__(self, index):
        image = self.images[index]
        emotion = self.emotions[index]

        return img_proc(image), emotion

    def __len__(self):
        return len(self.images)

train_loader = torch.utils.data.DataLoader(
    FERPlusDataset(
        image_path,
        training_emotion_path,
        'train'
    ),
    batch_size=batch_size,
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    FERPlusDataset(
        image_path,
        training_emotion_path,
        'test'
    ),
    batch_size=batch_size,
    shuffle=False,
)

def accuracy(output, target):
    batch_size = target.size(0)
    acc = 0
    for i in range(batch_size):
        true = target[i]
        pred = output[i]
        index_max = torch.argmax(pred)
        if true[index_max] == torch.max(true):
            acc += 1
    acc = float(acc)/batch_size
    return acc

In [4]:
def train(model,n_epochs=epochs, learningrate=lr, robust=False):
    # optimizer
    if robust:
        optimizer = RobustOptimizer(filter(lambda p: p.requires_grad, model.parameters()), optim.Adam, lr=learningrate)
    else:
        optimizer=optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learningrate)

    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()
        for data, label in tqdm(train_loader):
            data = data.to(device)
            label = label.to(device)

            output = model(data)
            loss = criterion(output, label)

            if robust:
                #optimizer.zero_grad()
                loss.backward()
                optimizer.first_step(zero_grad=True)
  
                # second forward-backward pass
                output = model(data)
                loss = criterion(output, label)
                loss.backward()
                optimizer.second_step(zero_grad=True)
            else:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            acc = (output.argmax(dim=1) == label.argmax(dim=1)).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in val_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = accuracy(val_output, label)
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += val_loss / len(val_loader)

        print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        #scheduler.step()
    
    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        model.eval()
        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for data, label in val_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = accuracy(val_output, label)
                epoch_val_accuracy += acc / len(val_loader)
                epoch_val_loss += val_loss / len(val_loader)

        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")

In [5]:
def set_parameter_requires_grad(model, requires_grad):
    for param in model.parameters():
        param.requires_grad = requires_grad

In [6]:
criterion = nn.MSELoss().to(device) 
model=timm.create_model('tf_efficientnet_b0_ns', pretrained=False)
model.classifier=torch.nn.Identity()
model.load_state_dict(torch.load('state_vggface2_enet0_new.pt'))

model.classifier=nn.Sequential(nn.Linear(in_features=1280, out_features=8))
model=model.to(device)

In [7]:
set_parameter_requires_grad(model, requires_grad=False)
set_parameter_requires_grad(model.classifier, requires_grad=True)
train(model, 10, 1e-3, robust=False)

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:24<00:00, 11.74it/s]


Epoch : 1 - loss : 0.0742 - acc: 0.6237 - val_loss : 0.0319 - val_acc: 0.7420



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:22<00:00, 12.04it/s]


Epoch : 2 - loss : 0.0692 - acc: 0.6554 - val_loss : 0.0309 - val_acc: 0.7460



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:22<00:00, 12.03it/s]


...

Epoch : 8 - loss : 0.0674 - acc: 0.6659 - val_loss : 0.0330 - val_acc: 0.7515



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:22<00:00, 12.09it/s]


Epoch : 9 - loss : 0.0670 - acc: 0.6698 - val_loss : 0.0316 - val_acc: 0.7419



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.71it/s]


Epoch : 10 - loss : 0.0671 - acc: 0.6683 - val_loss : 0.0298 - val_acc: 0.7583

Best acc:0.7720157657657657
val_loss : 0.0277 - val_acc: 0.7720



In [8]:
set_parameter_requires_grad(model, requires_grad=True)
train(model, 30, 1e-4, robust=False)

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [06:58<00:00,  2.38it/s]


Epoch : 1 - loss : 0.0581 - acc: 0.6999 - val_loss : 0.0181 - val_acc: 0.8259



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:34<00:00,  6.46it/s]


Epoch : 2 - loss : 0.0512 - acc: 0.7316 - val_loss : 0.0171 - val_acc: 0.8210



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:34<00:00,  6.46it/s]


...

Epoch : 28 - loss : 0.0059 - acc: 0.9746 - val_loss : 0.0286 - val_acc: 0.7714



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:34<00:00,  6.46it/s]


Epoch : 29 - loss : 0.0055 - acc: 0.9757 - val_loss : 0.0293 - val_acc: 0.7653



100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [02:34<00:00,  6.46it/s]


Epoch : 30 - loss : 0.0054 - acc: 0.9749 - val_loss : 0.0302 - val_acc: 0.7678

Best acc:0.8259009009009004
val_loss : 0.0181 - val_acc: 0.8259



In [9]:
torch.save(model.state_dict(), './models/mtcnn-8label-random-.pth')